
---

# Transformaciones
---


In [3]:
import pandas as pd
import ast
df = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\movies_dataset.csv', low_memory=False)
df2 = pd.read_csv('C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\Movies\\credits.csv', low_memory=False)

#'''trabajaré al mismo tiempo con el DF Credits, ya que algunas funciones de la API necesitan datos provenientes de este'''

In [4]:
#los convertimos a 'parquet' para optimizar el rendimineto
df.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')
# Guardar con compresión snappy
df2.to_parquet('data_compressed.parquet', engine='pyarrow', compression='snappy')



In [5]:
# Contar el número de filas duplicadas
duplicados = df.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados

print("Número de filas duplicadas eliminadas:", numero_de_duplicados)

#segundo DF
# Contar el número de filas duplicadas 
duplicados = df2.duplicated().sum()

# Eliminar filas duplicadas
df_sin_duplicados = df2.drop_duplicates()

# Almacenar el número de filas duplicadas en una variable, esto simplemente para verificar que si existían filas duplicadas
numero_de_duplicados = duplicados
print("Número de filas duplicadas eliminadas en el segundo DF:", numero_de_duplicados)

Número de filas duplicadas eliminadas: 17
Número de filas duplicadas eliminadas en el segundo DF: 37


In [6]:

'''Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.'''
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])


In [7]:
'''Los valores nulos del campo release date deben eliminarse.'''

df = df.dropna(subset=['release_date'])

In [8]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.'''

df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [9]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

#convertimos las fechas al datatype datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 
df['release_year'] = df['release_date'].dt.year

In [10]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''
# Convertir las columnas 'revenue' y 'budget' a tipo numérico
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
# Rellenar valores nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando revenue / budget
# Usar np.where para evitar división por 0
import numpy as np
df['return'] = np.where(df['budget'] == 0, 0, df['revenue'] / df['budget'])

In [12]:
''' Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una
lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API!
O bien buscar la manera de acceder a esos datos sin desanidarlos.'''
# Función para convertir cadenas a diccionarios o listas
def convertir_en_obj(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x  # Retorna la cadena si no es un diccionario válido
    return x

# Aplicar la conversión a las series 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_en_obj)
df['genres'] = df['genres'].apply(convertir_en_obj)
df['production_companies'] = df['production_companies'].apply(convertir_en_obj)
df['production_countries'] = df['production_countries'].apply(convertir_en_obj)
df['spoken_languages'] = df['spoken_languages'].apply(convertir_en_obj)

# Desanidar diccionarios
BTC_df = df['belongs_to_collection'].apply(pd.Series)
BTC_df = BTC_df.drop(columns = 0, errors='ignore')  # Eliminar columna de índice si existe
nuevos_nombres_BTC = ['BTCid', 'BTCname', 'BTCposter_path', 'BTCbackdrop_path']
BTC_df.columns = nuevos_nombres_BTC

# Desanidar listas
genres_df = df['genres'].apply(pd.Series)
production_companies_df = df['production_companies'].apply(pd.Series)
production_countries_df = df['production_countries'].apply(pd.Series)
spoken_languages_df = df['spoken_languages'].apply(pd.Series)

# Asegurarse de que no haya nombres de columnas duplicados
genres_df = genres_df.rename(columns=lambda x: f'genre_{x}')
production_companies_df = production_companies_df.rename(columns=lambda x: f'company_{x}')
production_countries_df = production_countries_df.rename(columns=lambda x: f'country_{x}')
spoken_languages_df = spoken_languages_df.rename(columns=lambda x: f'language_{x}')

# Eliminar las columnas originales
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'])

# Unir los DataFrames desanidado al DataFrame original
df = df.join(BTC_df)
df = df.join(production_companies_df)
df = df.join(production_countries_df)
df = df.join(spoken_languages_df)


C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:21: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  BTC_df = df['belongs_to_collection'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:27: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  genres_df = df['genres'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:28: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  production_companies_df = df['production_companies'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_12724\1652706673.py:29: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series


---

### Transformaciones sobre credits (API)
---


In [13]:
''' Al ser un DF muy grande y necesitar solo partes muy especificas de los diccionarios que componen las series del DF,
en vez de desanidarlo todo, utilizaremos otras tecnicas para extraer los datos que necesitamos, y con estos crearemos una nueva tabla,
esta misma contendra solamente los datos que necesitamos para las funciones de nuestra API '''

# Extraer los nombres de la columna 'cast'
names = df2['cast'].apply(lambda x: [item['name'] for item in ast.literal_eval(x) if 'name' in item])

# Convertir la serie 'names' en un DataFrame
names_df = pd.DataFrame(names.tolist())

# Añadir la palabra 'nombre' delante del nombre de cada serie
names_df.columns = ['nombre_' + str(col) for col in names_df.columns]

# Crear una serie que contenga únicamente el nombre del director
names_df['Nombre_Director'] = df2['crew'].apply(
    lambda x: next((item['name'] for item in ast.literal_eval(x) if item.get('job') == 'Director'), None)
)

# volvemos a añadir los ID a las filas, ya que los necesitaremos luego para hacer un "Join"
names_df['id'] = df2['id'].astype(int)





In [14]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')

In [15]:

#Hacemos un "Join" entre ambos DF
df = pd.merge(names_df,df, on='id')



---

### Exportación del Datafame

---


In [18]:
df.to_parquet('transformados.parquet', index=False)
pd.set_option('display.max_columns', None)


In [20]:
df.head(1)

,nombre_0,nombre_1,nombre_2,nombre_3,nombre_4,nombre_5,nombre_6,nombre_7,nombre_8,nombre_9,nombre_10,nombre_11,nombre_12,nombre_13,nombre_14,nombre_15,nombre_16,nombre_17,nombre_18,nombre_19,nombre_20,nombre_21,nombre_22,nombre_23,nombre_24,nombre_25,nombre_26,nombre_27,nombre_28,nombre_29,nombre_30,nombre_31,nombre_32,nombre_33,nombre_34,nombre_35,nombre_36,nombre_37,nombre_38,nombre_39,nombre_40,nombre_41,nombre_42,nombre_43,nombre_44,nombre_45,nombre_46,nombre_47,nombre_48,nombre_49,nombre_50,nombre_51,nombre_52,nombre_53,nombre_54,nombre_55,nombre_56,nombre_57,nombre_58,nombre_59,nombre_60,nombre_61,nombre_62,nombre_63,nombre_64,nombre_65,nombre_66,nombre_67,nombre_68,nombre_69,nombre_70,nombre_71,nombre_72,nombre_73,nombre_74,nombre_75,nombre_76,nombre_77,nombre_78,nombre_79,nombre_80,nombre_81,nombre_82,nombre_83,nombre_84,nombre_85,nombre_86,nombre_87,nombre_88,nombre_89,nombre_90,nombre_91,nombre_92,nombre_93,nombre_94,nombre_95,nombre_96,nombre_97,nombre_98,nombre_99,nombre_100,nombre_101,nombre_102,nombre_103,nombre_104,nombre_105,nombre_106,nombre_107,nombre_108,nombre_109,nombre_110,nombre_111,nombre_112,nombre_113,nombre_114,nombre_115,nombre_116,nombre_117,nombre_118,nombre_119,nombre_120,nombre_121,nombre_122,nombre_123,nombre_124,nombre_125,nombre_126,nombre_127,nombre_128,nombre_129,nombre_130,nombre_131,nombre_132,nombre_133,nombre_134,nombre_135,nombre_136,nombre_137,nombre_138,nombre_139,nombre_140,nombre_141,nombre_142,nombre_143,nombre_144,nombre_145,nombre_146,nombre_147,nombre_148,nombre_149,nombre_150,nombre_151,nombre_152,nombre_153,nombre_154,nombre_155,nombre_156,nombre_157,nombre_158,nombre_159,nombre_160,nombre_161,nombre_162,nombre_163,nombre_164,nombre_165,nombre_166,nombre_167,nombre_168,nombre_169,nombre_170,nombre_171,nombre_172,nombre_173,nombre_174,nombre_175,nombre_176,nombre_177,nombre_178,nombre_179,nombre_180,nombre_181,nombre_182,nombre_183,nombre_184,nombre_185,nombre_186,nombre_187,nombre_188,nombre_189,nombre_190,nombre_191,nombre_192,nombre_193,nombre_194,nombre_195,nombre_196,nombre_197,nombre_198,nombre_199,nombre_200,nombre_201,nombre_202,nombre_203,nombre_204,nombre_205,nombre_206,nombre_207,nombre_208,nombre_209,nombre_210,nombre_211,nombre_212,nombre_213,nombre_214,nombre_215,nombre_216,nombre_217,nombre_218,nombre_219,nombre_220,nombre_221,nombre_222,nombre_223,nombre_224,nombre_225,nombre_226,nombre_227,nombre_228,nombre_229,nombre_230,nombre_231,nombre_232,nombre_233,nombre_234,nombre_235,nombre_236,nombre_237,nombre_238,nombre_239,nombre_240,nombre_241,nombre_242,nombre_243,nombre_244,nombre_245,nombre_246,nombre_247,nombre_248,nombre_249,nombre_250,nombre_251,nombre_252,nombre_253,nombre_254,nombre_255,nombre_256,nombre_257,nombre_258,nombre_259,nombre_260,nombre_261,nombre_262,nombre_263,nombre_264,nombre_265,nombre_266,nombre_267,nombre_268,nombre_269,nombre_270,nombre_271,nombre_272,nombre_273,nombre_274,nombre_275,nombre_276,nombre_277,nombre_278,nombre_279,nombre_280,nombre_281,nombre_282,nombre_283,nombre_284,nombre_285,nombre_286,nombre_287,nombre_288,nombre_289,nombre_290,nombre_291,nombre_292,nombre_293,nombre_294,nombre_295,nombre_296,nombre_297,nombre_298,nombre_299,nombre_300,nombre_301,nombre_302,nombre_303,nombre_304,nombre_305,nombre_306,nombre_307,nombre_308,nombre_309,nombre_310,nombre_311,nombre_312,Nombre_Director,id,budget,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,return,BTCid,BTCname,BTCposter_path,BTCbackdrop_path,company_0,company_1,company_2,company_3,company_4,company_5,company_6,company_7,company_8,company_9,company_10,company_11,company_12,company_13,company_14,company_15,company_16,company_17,company_18,company_19,company_20,company_21,company_22,company_23,company_24,company_25,country_0,country_1,country_2,country_3,country_4,country_5,country_6,country_7,country_8,country_9,country_10,country_11,country_12,country_13,country_14,country_15,country_16,country_1


---

### Transformaciones para el sistema de recomendación (machine learning)
---


In [29]:
#estas transformaciones las hago ya habiendo casi finalizado el trabajo, para el sistema de recomendación
#He decidido que para este crearé un nuevo DF, el cual será una versión reducida del mismo, y que mantendrá unicamente las series que considere pertinentes

df = pd.read_parquet ("C:\\Users\\fede\\Desktop\\LABs 1\\Proyecto individual - Federico Piparo\\Transformaciones\\transformados.parquet")

In [30]:
df['combined_description'] = df['title'].astype(str) + ' ' + df['Nombre_Director'].astype(str) + ' ' + df[[col for col in df.columns if 'nombre' in col]].astype(str).agg(' '.join, axis=1)

In [34]:
df.head(1)

,title,budget,overview,popularity,revenue,runtime,vote_average,return,BTCname,combined_description
0,Toy Story,30000000.0,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,7.7,12.451801,Toy Story Collection,Toy Story John Lasseter Tom Hanks Tim Allen Do...


In [35]:
#vamos a reducir el tamaño del df en base a esta columna, ya que es la más importante en cuanto a nuestro proceso de machine learning
#para poder deployar el modelo es necesario reducir el tamaño de nuestro df

df = df.drop_duplicates(subset='title', keep='first')
# Reemplazar NaN en 'combined_description' con una cadena vacía
df['combined_description'] = df['combined_description'].fillna('')

# Definir la función para contar el número de caracteres en una cadena
def count_characters(text):
    return len(text)

# Aplicar la función para contar caracteres
df['char_count'] = df['combined_description'].apply(count_characters)

# Especificar el umbral de caracteres
umbral = 1660  # Ajusta este valor según sea necesario

# Filtrar las filas para conservar solo aquellas con un conteo de caracteres mayor o igual al umbral
df = df[df['char_count'] >= umbral]

# Verificar el tamaño después del filtrado
print(f"Número de filas después del filtrado: {len(df)}")

# Eliminar la columna de conteo de caracteres si ya no es necesaria
df = df.drop(columns=['char_count'])

Número de filas después del filtrado: 28643


In [37]:
#ahora me quedaré unicamente con las series que utilizaré para mi sistema de recomendación.
columnas_deseadas = ['title', 'budget', 'overview', 'popularity', 'revenue', 'runtime', 'vote_average', 'return', 'BTCname', 'combined_description']
df_ML = df[columnas_deseadas]

In [41]:
df_ML.reset_index(drop=True, inplace=True)

In [43]:
df_ML.to_parquet('df_ML.parquet')